In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//h1[@class="part-detail-title"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//p[@class="part-detail-title-number"]/strong/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//p[@class="item-description"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="primaryImage"]/@src')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//div[@class="attribute-container flex-container flex-wrap"]/div')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Title': title,
                                         'Description': description,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for i in range(int(len(list_div) / 2)):
                    df_temp.loc[0, ''.join(list_div[i*2].xpath('./descendant-or-self::*/text()')).strip()[:-1]] = ''.join(list_div[i*2+1].xpath('./descendant-or-self::*/text()')).strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：775

[ok] - https://www.summitracing.com/parts/ado-10-9289
[尝试次数：1] - [剩余数量：765] - [当前时间：17:59:30]

[ok] - https://www.summitracing.com/parts/ado-10-9290
[尝试次数：1] - [剩余数量：764] - [当前时间：17:59:31]

[ok] - https://www.summitracing.com/parts/aaf-all26240
[尝试次数：1] - [剩余数量：763] - [当前时间：17:59:31]

[ok] - https://www.summitracing.com/parts/ado-11562588
[尝试次数：1] - [剩余数量：762] - [当前时间：17:59:32]

[ok] - https://www.summitracing.com/parts/ado-11589253
[尝试次数：1] - [剩余数量：761] - [当前时间：17:59:32]

[ok] - https://www.summitracing.com/parts/ado-12640746
[尝试次数：1] - [剩余数量：760] - [当前时间：17:59:32]

[ok] - https://www.summitracing.com/parts/ado-11570082
[尝试次数：1] - [剩余数量：759] - [当前时间：17:59:32]

[ok] - https://www.summitracing.com/parts/ado-10-9246
[尝试次数：1] - [剩余数量：758] - [当前时间：17:59:32]

[ok] - https://www.summitracing.com/parts/aaf-all26241
[尝试次数：1] - [剩余数量：757] - [当前时间：17:59:33]

[ok] - https://www.summitracing.com/parts/ado-24239049
[尝试次数：1] - [剩余数量：756] - [当前时间：17:59:33]

[ok] - https://www.summitracing.co

[ok] - https://www.summitracing.com/parts/cmb-08-0158
[尝试次数：2] - [剩余数量：678] - [当前时间：17:59:59]

[ok] - https://www.summitracing.com/parts/cmb-08-0157
[尝试次数：2] - [剩余数量：677] - [当前时间：17:59:59]

[ok] - https://www.summitracing.com/parts/cmb-08-0161
[尝试次数：1] - [剩余数量：676] - [当前时间：17:59:59]

[ok] - https://www.summitracing.com/parts/cmb-08-0159
[尝试次数：3] - [剩余数量：675] - [当前时间：18:00:00]

[ok] - https://www.summitracing.com/parts/cmb-08-0163
[尝试次数：5] - [剩余数量：674] - [当前时间：18:00:01]

[ok] - https://www.summitracing.com/parts/cmb-08-0153
[尝试次数：4] - [剩余数量：673] - [当前时间：18:00:01]

[ok] - https://www.summitracing.com/parts/cmb-08-0160
[尝试次数：6] - [剩余数量：672] - [当前时间：18:00:01]

[ok] - https://www.summitracing.com/parts/cmb-08-0165
[尝试次数：3] - [剩余数量：671] - [当前时间：18:00:01]

[ok] - https://www.summitracing.com/parts/cmb-08-0167
[尝试次数：4] - [剩余数量：670] - [当前时间：18:00:02]

[ok] - https://www.summitracing.com/parts/cmb-08-0170
[尝试次数：4] - [剩余数量：669] - [当前时间：18:00:03]

[ok] - https://www.summitracing.com/parts/cmb-08-0

[ok] - https://www.summitracing.com/parts/drt-31947142
[尝试次数：1] - [剩余数量：591] - [当前时间：18:00:38]

[ok] - https://www.summitracing.com/parts/drt-31947122
[尝试次数：2] - [剩余数量：590] - [当前时间：18:00:38]

[ok] - https://www.summitracing.com/parts/drt-31947211
[尝试次数：1] - [剩余数量：589] - [当前时间：18:00:39]

[ok] - https://www.summitracing.com/parts/drt-31947221
[尝试次数：1] - [剩余数量：588] - [当前时间：18:00:39]

[ok] - https://www.summitracing.com/parts/drt-31947212
[尝试次数：1] - [剩余数量：587] - [当前时间：18:00:39]

[ok] - https://www.summitracing.com/parts/drt-31947111
[尝试次数：8] - [剩余数量：586] - [当前时间：18:00:39]

[ok] - https://www.summitracing.com/parts/drt-31947222
[尝试次数：2] - [剩余数量：585] - [当前时间：18:00:40]

[ok] - https://www.summitracing.com/parts/ear-1136erl
[尝试次数：1] - [剩余数量：584] - [当前时间：18:00:40]

[ok] - https://www.summitracing.com/parts/drt-31947242
[尝试次数：1] - [剩余数量：583] - [当前时间：18:00:41]

[ok] - https://www.summitracing.com/parts/drt-62230001
[尝试次数：1] - [剩余数量：582] - [当前时间：18:00:41]

[ok] - https://www.summitracing.com/parts

[ok] - https://www.summitracing.com/parts/hly-302-3bk
[尝试次数：5] - [剩余数量：503] - [当前时间：18:01:11]

[ok] - https://www.summitracing.com/parts/hly-302-4
[尝试次数：6] - [剩余数量：502] - [当前时间：18:01:11]

[ok] - https://www.summitracing.com/parts/hly-302-10
[尝试次数：20] - [剩余数量：501] - [当前时间：18:01:11]

[ok] - https://www.summitracing.com/parts/hok-12616hkr-1
[尝试次数：4] - [剩余数量：500] - [当前时间：18:01:12]

[ok] - https://www.summitracing.com/parts/hme-vmt23mc
[尝试次数：3] - [剩余数量：499] - [当前时间：18:01:12]

[ok] - https://www.summitracing.com/parts/hme-vmt23m
[尝试次数：5] - [剩余数量：498] - [当前时间：18:01:12]

[ok] - https://www.summitracing.com/parts/hok-71222012hkr
[尝试次数：1] - [剩余数量：497] - [当前时间：18:01:12]

[ok] - https://www.summitracing.com/parts/hok-71222016hkr
[尝试次数：1] - [剩余数量：496] - [当前时间：18:01:13]

[ok] - https://www.summitracing.com/parts/hly-302-35
[尝试次数：4] - [剩余数量：495] - [当前时间：18:01:14]

[ok] - https://www.summitracing.com/parts/hok-71222013hkr
[尝试次数：6] - [剩余数量：494] - [当前时间：18:01:14]

[ok] - https://www.summitracing.com/par

[ok] - https://www.summitracing.com/parts/lim-2258
[尝试次数：10] - [剩余数量：416] - [当前时间：18:01:50]

[ok] - https://www.summitracing.com/parts/lss-gmls2002b
[尝试次数：7] - [剩余数量：415] - [当前时间：18:01:50]

[ok] - https://www.summitracing.com/parts/lim-20228
[尝试次数：2] - [剩余数量：414] - [当前时间：18:01:51]

[ok] - https://www.summitracing.com/parts/irp-egm-740c
[尝试次数：18] - [剩余数量：413] - [当前时间：18:01:51]

[ok] - https://www.summitracing.com/parts/lss-gmls2004m
[尝试次数：2] - [剩余数量：412] - [当前时间：18:01:51]

[ok] - https://www.summitracing.com/parts/lim-2259
[尝试次数：1] - [剩余数量：411] - [当前时间：18:01:52]

[ok] - https://www.summitracing.com/parts/lss-gmls2004b
[尝试次数：3] - [剩余数量：410] - [当前时间：18:01:52]

[ok] - https://www.summitracing.com/parts/lss-gmls4002
[尝试次数：1] - [剩余数量：409] - [当前时间：18:01:53]

[ok] - https://www.summitracing.com/parts/lss-gmls2006m
[尝试次数：1] - [剩余数量：408] - [当前时间：18:01:54]

[ok] - https://www.summitracing.com/parts/lss-gmls4009
[尝试次数：3] - [剩余数量：407] - [当前时间：18:01:54]

[ok] - https://www.summitracing.com/parts/lss

[ok] - https://www.summitracing.com/parts/mll-s0002
[尝试次数：1] - [剩余数量：330] - [当前时间：18:02:28]

[ok] - https://www.summitracing.com/parts/mml-oslsx401111
[尝试次数：1] - [剩余数量：329] - [当前时间：18:02:28]

[ok] - https://www.summitracing.com/parts/mml-oslsx401015
[尝试次数：1] - [剩余数量：328] - [当前时间：18:02:28]

[ok] - https://www.summitracing.com/parts/mll-psls4272sctk
[尝试次数：1] - [剩余数量：327] - [当前时间：18:02:29]

[ok] - https://www.summitracing.com/parts/mml-oslsx401120
[尝试次数：1] - [剩余数量：326] - [当前时间：18:02:29]

[ok] - https://www.summitracing.com/parts/mml-oslsx401014
[尝试次数：1] - [剩余数量：325] - [当前时间：18:02:30]

[ok] - https://www.summitracing.com/parts/mll-s0005
[尝试次数：3] - [剩余数量：324] - [当前时间：18:02:31]

[ok] - https://www.summitracing.com/parts/lss-gmls9017
[尝试次数：8] - [剩余数量：323] - [当前时间：18:02:32]

[ok] - https://www.summitracing.com/parts/mor-20142
[尝试次数：1] - [剩余数量：322] - [当前时间：18:02:32]

[ok] - https://www.summitracing.com/parts/mor-20105
[尝试次数：6] - [剩余数量：321] - [当前时间：18:02:32]

[ok] - https://www.summitracing.com/

[ok] - https://www.summitracing.com/parts/nal-19433059
[尝试次数：3] - [剩余数量：243] - [当前时间：18:03:09]

[ok] - https://www.summitracing.com/parts/nal-19435108
[尝试次数：2] - [剩余数量：242] - [当前时间：18:03:09]

[ok] - https://www.summitracing.com/parts/nal-19432629
[尝试次数：3] - [剩余数量：241] - [当前时间：18:03:09]

[ok] - https://www.summitracing.com/parts/nal-19434600
[尝试次数：1] - [剩余数量：240] - [当前时间：18:03:10]

[ok] - https://www.summitracing.com/parts/nal-24203590
[尝试次数：3] - [剩余数量：239] - [当前时间：18:03:11]

[ok] - https://www.summitracing.com/parts/nal-19435110
[尝试次数：2] - [剩余数量：238] - [当前时间：18:03:11]

[ok] - https://www.summitracing.com/parts/nal-s2023-22
[尝试次数：1] - [剩余数量：237] - [当前时间：18:03:11]

[ok] - https://www.summitracing.com/parts/nal-24203911
[尝试次数：4] - [剩余数量：236] - [当前时间：18:03:12]

[ok] - https://www.summitracing.com/parts/nal-s2023-23
[尝试次数：2] - [剩余数量：235] - [当前时间：18:03:12]

[ok] - https://www.summitracing.com/parts/nal-s2023-32
[尝试次数：1] - [剩余数量：234] - [当前时间：18:03:13]

[ok] - https://www.summitracing.com/part

[ok] - https://www.summitracing.com/parts/rnb-69011
[尝试次数：1] - [剩余数量：157] - [当前时间：18:03:51]

[ok] - https://www.summitracing.com/parts/rdr-3310694
[尝试次数：11] - [剩余数量：156] - [当前时间：18:03:51]

[ok] - https://www.summitracing.com/parts/rnb-700-365
[尝试次数：1] - [剩余数量：155] - [当前时间：18:03:51]

[ok] - https://www.summitracing.com/parts/rnb-092-018
[尝试次数：1] - [剩余数量：154] - [当前时间：18:03:52]

[ok] - https://www.summitracing.com/parts/rnb-090-053cd
[尝试次数：3] - [剩余数量：153] - [当前时间：18:03:52]

[ok] - https://www.summitracing.com/parts/rnb-963-058
[尝试次数：1] - [剩余数量：152] - [当前时间：18:03:52]

[ok] - https://www.summitracing.com/parts/rnb-69010
[尝试次数：1] - [剩余数量：151] - [当前时间：18:03:52]

[ok] - https://www.summitracing.com/parts/rnb-963-058d
[尝试次数：1] - [剩余数量：150] - [当前时间：18:03:52]

[ok] - https://www.summitracing.com/parts/sgt-gmp53a
[尝试次数：1] - [剩余数量：149] - [当前时间：18:03:54]

[ok] - https://www.summitracing.com/parts/skm-ds01a
[尝试次数：1] - [剩余数量：148] - [当前时间：18:03:54]

[ok] - https://www.summitracing.com/parts/skm-g01
[尝试

[ok] - https://www.summitracing.com/parts/trd-42036
[尝试次数：1] - [剩余数量：69] - [当前时间：18:04:27]

[ok] - https://www.summitracing.com/parts/trd-4207
[尝试次数：4] - [剩余数量：68] - [当前时间：18:04:27]

[ok] - https://www.summitracing.com/parts/trd-42164
[尝试次数：2] - [剩余数量：67] - [当前时间：18:04:28]

[ok] - https://www.summitracing.com/parts/trd-42162
[尝试次数：2] - [剩余数量：66] - [当前时间：18:04:28]

[ok] - https://www.summitracing.com/parts/trd-42034
[尝试次数：9] - [剩余数量：65] - [当前时间：18:04:29]

[ok] - https://www.summitracing.com/parts/trd-42211
[尝试次数：2] - [剩余数量：64] - [当前时间：18:04:29]

[ok] - https://www.summitracing.com/parts/trd-42043
[尝试次数：7] - [剩余数量：63] - [当前时间：18:04:29]

[ok] - https://www.summitracing.com/parts/trd-42213
[尝试次数：1] - [剩余数量：62] - [当前时间：18:04:30]

[ok] - https://www.summitracing.com/parts/trd-42214
[尝试次数：1] - [剩余数量：61] - [当前时间：18:04:31]

[ok] - https://www.summitracing.com/parts/trd-42221
[尝试次数：2] - [剩余数量：60] - [当前时间：18:04:31]

[ok] - https://www.summitracing.com/parts/trd-42163
[尝试次数：10] - [剩余数量：59] - [当前时间